In [2]:
import numpy as np
import os
from tqdm import tqdm

import OpenEXR
import Imath
from load_exr import read_exr_as_np

In [10]:
def func_fitting(OBJECT, split, view_no, deg):
    waves = [450, 500, 550, 600, 650]
    data = []
    waves_samples = range(450, 650+1, 10)
    
    for w in waves:
        exr_path = f"C:/Users/owner/Desktop/{OBJECT}/{w}/{split}/r_{view_no}.exr"
        exr_path = f"C:/Users/owner/Desktop/{OBJECT}/{w}/r_{view_no}.exr"
        
        exr_data = read_exr_as_np(exr_path)[0]
        
        W, H = exr_data.shape[:2]
        data.append(exr_data)
        
    stokes = []
    
    
    for k in tqdm([4, 7, 10, 13], desc='fit'):
        
        array = np.zeros((W, H, len(waves_samples))).astype(np.float32)
        
        for ii in range(H*W):
            i, j = ii//W, ii%W
            # for j in range(W):
            xdata = np.array(waves)
            ydata = np.array([data[0][i][j][k], data[1][i][j][k], data[2][i][j][k], data[3][i][j][k], data[4][i][j][k]])
            poly = np.polyfit(xdata, ydata, deg)
            
            for w_i, w in enumerate(waves_samples):
                array[i,j,w_i] = np.polyval(poly, w)
                
        stokes.append(array)
        
    # 4 x H x W x wavelength
    for w_i, wavelength in enumerate(waves_samples):
        save_path = f"//jarvis.postech.ac.kr/workspace/youngchan/nepf/data/exr_fit/{OBJECT}/{split}/{wavelength}"
        os.makedirs(save_path, exist_ok=True)
        
        fname = os.path.join(save_path, f"r_{view_no}.exr")
        s0, s1, s2, s3 = stokes[0][...,w_i], stokes[1][...,w_i], stokes[2][...,w_i], stokes[3][...,w_i]
        
        h = OpenEXR.Header(s0.shape[1], s0.shape[0])
        ctype = Imath.Channel(Imath.PixelType(OpenEXR.FLOAT))
        h["channels"] = {"s0":ctype, "s1":ctype, "s2":ctype, "s3":ctype}
        
        exr = OpenEXR.OutputFile(fname, h)
        exr.writePixels({'s0': s0.tostring(), 's1': s1.tostring(), 's2': s2.tostring(),  's3':s3.tostring()})
        exr.close()